# PyODPS to Alink & Mars

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# PyODPS

In [2]:
# PyODPS
# pip install -U pyodps
from odps import __version__
print(__version__)

0.8.3


In [3]:
from odps import ODPS
from odps.df import DataFrame
from odps import options
options.df.odps.sort.limit = 1000000000

## 配置连接

In [4]:
o = ODPS(access_id='your_access_id', 
         secret_access_key='your_secret_access_key', 
         project='your_project_name', 
         end_point='https://service.odps.aliyun.com/api'
         )

## 读取数据

In [5]:
autoMPG = DataFrame(o.get_table('autoMPG', project = "your_project_name"))
autoMPG.count()

ODPSError: AccessKeyIdNotFound: RequestId: 5E05EA196CBF97531346AE1A
ODPS-0410051:Invalid credentials - accessKeyId not found: LTAI4Fc7aj9bKGiaQY8HE8VX

In [6]:
autoMPG

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130,3504,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165,3693,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150,3436,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150,3433,12.0,70,1,"""amc rebel sst"""
4,17.0,8,302.0,140,3449,10.5,70,1,"""ford torino"""
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1,"""ford mustang gl"""
394,44.0,4,97.0,52,2130,24.6,82,2,"""vw pickup"""
395,32.0,4,135.0,84,2295,11.6,82,1,"""dodge rampage"""
396,28.0,4,120.0,79,2625,18.6,82,1,"""ford ranger"""


## 数据处理

In [14]:
#去除掉horsepower列中值为？的记录
dfAuto = autoMPG[autoMPG.horsepower != '?']
dfAuto.count()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

392

In [15]:
dfAuto.schema

odps.Schema {
  mpg                     float64       
  cylinders               int64         
  displacement            float64       
  horsepower              string        
  weight                  int64         
  acceleration            float64       
  model_year              int64         
  origin                  int64         
  car_name                string        
}

In [16]:
dfCYL = dfAuto.groupby('cylinders').agg(maxMPG = dfAuto.mpg.max(), \
                                        minMPG = dfAuto.mpg.min(), \
                                        cylindersCnt = dfAuto.cylinders.count()).sort('cylinders', \
                                                                                        ascending = False)
dfCYL

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

,cylinders,cylindersCnt,maxMPG,minMPG
0,8,103,26.6,9.0
1,6,83,38.0,15.0
2,5,3,36.4,20.3
3,4,199,46.6,18.0
4,3,4,23.7,18.0


In [17]:
#去除掉气缸个数为3的记录
dfCYL0 = dfAuto[dfAuto.cylinders != 3]
dfCYL0
dfCYL0.count()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

388

In [18]:
#去除掉气缸个数为5的记录
dfCYL1 = dfCYL0[dfCYL0.cylinders != 5]
dfCYL1
dfCYL1.count()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

385

In [19]:
#去除掉气缸个数为6的记录
dfCYL2 = dfCYL1[dfCYL1.cylinders !=6]
dfCYL2
dfCYL2.count()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

302

In [20]:
dfCYL2

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130,3504,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165,3693,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150,3436,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150,3433,12.0,70,1,"""amc rebel sst"""
4,17.0,8,302.0,140,3449,10.5,70,1,"""ford torino"""
...,...,...,...,...,...,...,...,...,...
297,27.0,4,140.0,86,2790,15.6,82,1,"""ford mustang gl"""
298,44.0,4,97.0,52,2130,24.6,82,2,"""vw pickup"""
299,32.0,4,135.0,84,2295,11.6,82,1,"""dodge rampage"""
300,28.0,4,120.0,79,2625,18.6,82,1,"""ford ranger"""


In [22]:
#将horsepower字段的类型改为int型，并去除字符串类型字段car_name
dfCYL2['horsepower'] = dfCYL2['horsepower'].astype('int64')
dfNum = dfCYL2.exclude('car_name')
dfNum.schema

odps.Schema {
  mpg                     float64       
  cylinders               int64         
  displacement            float64       
  horsepower              int64         
  weight                  int64         
  acceleration            float64       
  model_year              int64         
  origin                  int64         
}

In [23]:
#设置字段连续性及标签字段
dfNew = dfNum.continuous('mpg', 'displacement', 'horsepower', 'weight', \
                         'acceleration', 'model_year', 'origin', 'cylinders' ).roles(label='cylinders')

In [24]:
#归一化，并划分训练集和测试集
fitData, testData = dfNew.min_max_scale(['mpg', 'displacement', 'horsepower', 'weight', \
                                    'acceleration', 'model_year', 'origin']).split(0.7)

In [25]:
dfNew.dtypes

odps.Schema {
  mpg                     float64        FEATURE
  cylinders               int64          LABEL
  displacement            float64        FEATURE
  horsepower              int64          FEATURE
  weight                  int64          FEATURE
  acceleration            float64        FEATURE
  model_year              int64          FEATURE
  origin                  int64          FEATURE
}

In [26]:
fitData.dtypes

odps.Schema {
  mpg                     float64        FEATURE
  cylinders               int64          LABEL
  displacement            float64        FEATURE
  horsepower              float64        FEATURE
  weight                  float64        FEATURE
  acceleration            float64        FEATURE
  model_year              float64        FEATURE
  origin                  float64        FEATURE
}

In [27]:
testData.dtypes

odps.Schema {
  mpg                     float64        FEATURE
  cylinders               int64          LABEL
  displacement            float64        FEATURE
  horsepower              float64        FEATURE
  weight                  float64        FEATURE
  acceleration            float64        FEATURE
  model_year              float64        FEATURE
  origin                  float64        FEATURE
}

In [28]:
fitData

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,0.159574,8,0.728682,0.646739,0.589736,0.208333,0.0,0.0
1,0.239362,8,0.645995,0.565217,0.516870,0.178571,0.0,0.0
2,0.186170,8,0.609819,0.565217,0.516019,0.238095,0.0,0.0
3,0.212766,8,0.604651,0.510870,0.520556,0.148810,0.0,0.0
4,0.159574,8,0.932817,0.826087,0.773462,0.119048,0.0,0.0
...,...,...,...,...,...,...,...,...
206,0.478723,4,0.214470,0.239130,0.379076,0.553571,1.0,0.0
207,0.478723,4,0.186047,0.217391,0.333711,0.452381,1.0,0.0
208,0.930851,4,0.074935,0.032609,0.146583,0.988095,1.0,0.5
209,0.611702,4,0.173127,0.206522,0.193365,0.214286,1.0,0.0


In [29]:
type(fitData)

odps.df.expr.core.SplitCollectionExpr

In [31]:
pdfTrain = fitData.to_pandas()
pdfTrain

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,0.159574,8,0.728682,0.646739,0.589736,0.208333,0.0,0.0
1,0.239362,8,0.645995,0.565217,0.516870,0.178571,0.0,0.0
2,0.186170,8,0.609819,0.565217,0.516019,0.238095,0.0,0.0
3,0.212766,8,0.604651,0.510870,0.520556,0.148810,0.0,0.0
4,0.159574,8,0.932817,0.826087,0.773462,0.119048,0.0,0.0
...,...,...,...,...,...,...,...,...
206,0.478723,4,0.214470,0.239130,0.379076,0.553571,1.0,0.0
207,0.478723,4,0.186047,0.217391,0.333711,0.452381,1.0,0.0
208,0.930851,4,0.074935,0.032609,0.146583,0.988095,1.0,0.5
209,0.611702,4,0.173127,0.206522,0.193365,0.214286,1.0,0.0


In [32]:
type(pdfTrain)

pandas.core.frame.DataFrame

# Mars

In [33]:
# Mars
# pip uninstall pymars
# pip install --pre pymars
from mars import __version__
print(__version__)

0.3.0rc1


In [34]:
import mars.tensor as mt
import mars.dataframe as md
from mars.learn.contrib.xgboost import XGBClassifier
from mars.config import options
options.eager_mode = True

## Pandas DataFrame -> Mars Tensor

In [35]:
# pandas dataframe to tensor 
mtTrain = md.DataFrame(pdfTrain).to_tensor()
mtTrain

Tensor <op=TensorFromDataFrame, shape=(211, 8), key=12dcaf693a53dde759e87f02635bea1a, data=
array([[0.15957447, 8.        , 0.72868217, ..., 0.20833333, 0.        ,
        0.        ],
       [0.2393617 , 8.        , 0.64599483, ..., 0.17857143, 0.        ,
        0.        ],
       [0.18617021, 8.        , 0.60981912, ..., 0.23809524, 0.        ,
        0.        ],
       ...,
       [0.93085106, 4.        , 0.0749354 , ..., 0.98809524, 1.        ,
        0.5       ],
       [0.61170213, 4.        , 0.17312661, ..., 0.21428571, 1.        ,
        0.        ],
       [0.58510638, 4.        , 0.13178295, ..., 0.67857143, 1.        ,
        0.        ]])>

In [36]:
type(mtTrain)

mars.tensor.core.Tensor

## Mars Tensor -> Pandas DataFrame

In [37]:
mdfTrain_2 = md.DataFrame(mtTrain)
mdfTrain_2

DataFrame <op=DataFrameFromTensor, key=51548439c5d3223ad636d3f9425e2044, data=
            0    1         2         3         4         5    6    7
0    0.159574  8.0  0.728682  0.646739  0.589736  0.208333  0.0  0.0
1    0.239362  8.0  0.645995  0.565217  0.516870  0.178571  0.0  0.0
2    0.186170  8.0  0.609819  0.565217  0.516019  0.238095  0.0  0.0
3    0.212766  8.0  0.604651  0.510870  0.520556  0.148810  0.0  0.0
4    0.159574  8.0  0.932817  0.826087  0.773462  0.119048  0.0  0.0
..        ...  ...       ...       ...       ...       ...  ...  ...
206  0.478723  4.0  0.214470  0.239130  0.379076  0.553571  1.0  0.0
207  0.478723  4.0  0.186047  0.217391  0.333711  0.452381  1.0  0.0
208  0.930851  4.0  0.074935  0.032609  0.146583  0.988095  1.0  0.5
209  0.611702  4.0  0.173127  0.206522  0.193365  0.214286  1.0  0.0
210  0.585106  4.0  0.131783  0.195652  0.313864  0.678571  1.0  0.0

[211 rows x 8 columns]>

In [38]:
type(mdfTrain_2)

mars.dataframe.initializer.DataFrame

In [39]:
pdfTrain_2 = mdfTrain_2.execute()
pdfTrain_2

,0,1,2,3,4,5,6,7
0,0.159574,8.0,0.728682,0.646739,0.589736,0.208333,0.0,0.0
1,0.239362,8.0,0.645995,0.565217,0.516870,0.178571,0.0,0.0
2,0.186170,8.0,0.609819,0.565217,0.516019,0.238095,0.0,0.0
3,0.212766,8.0,0.604651,0.510870,0.520556,0.148810,0.0,0.0
4,0.159574,8.0,0.932817,0.826087,0.773462,0.119048,0.0,0.0
...,...,...,...,...,...,...,...,...
206,0.478723,4.0,0.214470,0.239130,0.379076,0.553571,1.0,0.0
207,0.478723,4.0,0.186047,0.217391,0.333711,0.452381,1.0,0.0
208,0.930851,4.0,0.074935,0.032609,0.146583,0.988095,1.0,0.5
209,0.611702,4.0,0.173127,0.206522,0.193365,0.214286,1.0,0.0


In [40]:
type(pdfTrain_2)

pandas.core.frame.DataFrame

## Mars Tensor -> Numpy Array

In [41]:
npTrain = mtTrain.execute()
npTrain

array([[0.15957447, 8.        , 0.72868217, ..., 0.20833333, 0.        ,
        0.        ],
       [0.2393617 , 8.        , 0.64599483, ..., 0.17857143, 0.        ,
        0.        ],
       [0.18617021, 8.        , 0.60981912, ..., 0.23809524, 0.        ,
        0.        ],
       ...,
       [0.93085106, 4.        , 0.0749354 , ..., 0.98809524, 1.        ,
        0.5       ],
       [0.61170213, 4.        , 0.17312661, ..., 0.21428571, 1.        ,
        0.        ],
       [0.58510638, 4.        , 0.13178295, ..., 0.67857143, 1.        ,
        0.        ]])

In [42]:
type(npTrain)

numpy.ndarray

## Numpy Array -> Mars Tensor

In [43]:
mtTrain_2 = mt.array(npTrain)
mtTrain_2

Tensor <op=ArrayDataSource, shape=(211, 8), key=53a7857fd9bc7f092815900bfbbf3c01, data=
array([[0.15957447, 8.        , 0.72868217, ..., 0.20833333, 0.        ,
        0.        ],
       [0.2393617 , 8.        , 0.64599483, ..., 0.17857143, 0.        ,
        0.        ],
       [0.18617021, 8.        , 0.60981912, ..., 0.23809524, 0.        ,
        0.        ],
       ...,
       [0.93085106, 4.        , 0.0749354 , ..., 0.98809524, 1.        ,
        0.5       ],
       [0.61170213, 4.        , 0.17312661, ..., 0.21428571, 1.        ,
        0.        ],
       [0.58510638, 4.        , 0.13178295, ..., 0.67857143, 1.        ,
        0.        ]])>

In [44]:
type(mtTrain_2)

mars.tensor.core.Tensor

# Alink

## 环境准备

In [9]:
# set env 
from pyalink.alink import *
import sys, os
resetEnv()
# Local
useLocalEnv(2, config=None)

JavaObject id=o47

## 数据准备

In [7]:
# schema of train data
schemaStr = "id string, click string, dt string, C1 string, banner_pos int, site_id string, \
            site_domain string, site_category string, app_id string, app_domain string, \
            app_category string, device_id string, device_ip string, device_model string, \
            device_type string, device_conn_type string, C14 int, C15 int, C16 int, C17 int, \
            C18 int, C19 int, C20 int, C21 int"

# prepare batch train data
batchTrainDataFn = "http://alink-release.oss-cn-beijing.aliyuncs.com/data-files/avazu-small.csv"
trainBatchData = CsvSourceBatchOp().setFilePath(batchTrainDataFn) \
        .setSchemaStr(schemaStr) \
        .setIgnoreFirstLine(True);
# feature fit
labelColName = "click"
vecColName = "vec"
numHashFeatures = 30000
selectedColNames =["C1","banner_pos","site_category","app_domain",
                  "app_category","device_type","device_conn_type", 
                  "C14","C15","C16","C17","C18","C19","C20","C21",
                   "site_id","site_domain","device_id","device_model"]

categoryColNames = ["C1","banner_pos","site_category","app_domain", 
                    "app_category","device_type","device_conn_type",
                    "site_id","site_domain","device_id","device_model"]

numericalColNames = ["C14","C15","C16","C17","C18","C19","C20","C21"]

# prepare stream train data
wholeDataFile = "http://alink-release.oss-cn-beijing.aliyuncs.com/data-files/avazu-ctr-train-8M.csv"
data = CsvSourceStreamOp() \
        .setFilePath(wholeDataFile) \
        .setSchemaStr(schemaStr) \
        .setIgnoreFirstLine(True);

# split stream to train and eval data
spliter = SplitStreamOp().setFraction(0.5).linkFrom(data)
train_stream_data = spliter
test_stream_data = spliter.getSideOutput(0)

## 在线学习五步骤
<ul>
    <li>步骤一、特征工程</li>
    <li>步骤二、批式模型训练</li>
    <li>步骤三、在线模型训练（FTRL）</li>
    <li>步骤四、在线预测</li>
    <li>步骤五、在线评估</li>
</ul>

### 步骤一、特征工程

In [10]:
# setup feature enginerring pipeline
feature_pipeline = Pipeline() \
        .add(StandardScaler() \
                .setSelectedCols(numericalColNames)) \
        .add(FeatureHasher() \
                .setSelectedCols(selectedColNames) \
                .setCategoricalCols(categoryColNames) \
                .setOutputCol(vecColName) \
                .setNumFeatures(numHashFeatures))

# fit and save feature pipeline model
FEATURE_PIPELINE_MODEL_FILE = os.path.join(os.getcwd(), "feature_pipe_model.csv")
feature_pipeline.fit(trainBatchData).save(FEATURE_PIPELINE_MODEL_FILE);

BatchOperator.execute();

# load pipeline model
feature_pipelineModel = PipelineModel.load(FEATURE_PIPELINE_MODEL_FILE);


### 步骤二、批式模型训练

In [11]:
# train initial batch model
lr = LogisticRegressionTrainBatchOp()
initModel = lr.setVectorCol(vecColName) \
        .setLabelCol(labelColName) \
        .setWithIntercept(True) \
        .setMaxIter(10) \
        .linkFrom(feature_pipelineModel.transform(trainBatchData)) 

### 步骤三、在线模型训练（FTRL）

In [12]:
# ftrl train 
model = FtrlTrainStreamOp(initModel) \
        .setVectorCol(vecColName) \
        .setLabelCol(labelColName) \
        .setWithIntercept(True) \
        .setAlpha(0.1) \
        .setBeta(0.1) \
        .setL1(0.01) \
        .setL2(0.01) \
        .setTimeInterval(10) \
        .setVectorSize(numHashFeatures) \
        .linkFrom(feature_pipelineModel.transform(train_stream_data))

### 步骤四、在线预测

In [ ]:
# ftrl predict
predResult = FtrlPredictStreamOp(initModel) \
        .setVectorCol(vecColName) \
        .setPredictionCol("pred") \
        .setReservedCols([labelColName]) \
        .setPredictionDetailCol("details") \
        .linkFrom(model, feature_pipelineModel.transform(test_stream_data))

predResult.print(key="predResult", refreshInterval = 30, maxLimit=20)

'DataStream predResult: (Updated on 2019-12-27 23:57:49)'

,click,pred,details


### 步骤五、在线评估

In [ ]:
# ftrl eval
EvalBinaryClassStreamOp() \
        .setLabelCol(labelColName) \
        .setPredictionCol("pred") \
        .setPredictionDetailCol("details") \
        .setTimeInterval(10) \
        .linkFrom(predResult) \
        .link(JsonValueStreamOp() \
                .setSelectedCol("Data") \
                .setReservedCols(["Statistics"]) \
                .setOutputCols(["Accuracy", "AUC", "ConfusionMatrix"]) \
                .setJsonPath(["$.Accuracy", "$.AUC", "$.ConfusionMatrix"])) \
                .print(key="evaluation", refreshInterval = 30, maxLimit=20)
StreamOperator.execute();

'DataStream evaluation: (Updated on 2019-12-27 23:57:41)'

,Statistics,Accuracy,AUC,ConfusionMatrix
0,all,0.8391606650616757,0.7359426635233952,"[[45557,37260],[589399,3223964]]"
1,window,0.8204482799952055,0.7134813769518872,"[[282,246],[4248,20253]]"
2,all,0.8390412242754722,0.7358106990035772,"[[45839,37506],[593647,3244217]]"
3,window,0.8258867667121419,0.7249371775942718,"[[380,298],[4807,23835]]"
4,all,0.8389435946426416,0.7357393208830327,"[[46219,37804],[598454,3268052]]"
5,all,0.8388325083774235,0.7356336212105179,"[[46531,38046],[602686,3288303]]"


## End